In [18]:
import os
import pandas as pd
import numpy as np
import math
from _datetime import datetime, date, timedelta

##### reference: https://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf 

In [19]:
query = '''
    query {
        ticks(
            first: 1000,
            orderBy: tickIdx,
            orderDirection: asc,
            where: {
                poolAddress: '0x99ac8ca7087fa4a2a1fb6357269965a2014abc35'
            }
        ) {
            tickIdx
            liquidityNet
        }
    }
''' 

In [20]:
# Read data
df = pd.read_excel('/Users/wanjinwoo/Desktop/Antalpha/Github/data/poolday.xlsx')

df = df.rename(columns={ 'Liquidity': 'L', 'Tick': 'currTick', 'Token1 Price (Excel)': 'currPrice', 'Volume USD (Excel)': 'daily_volume', 'TVL USD (Excel)': 'TVL' })

df = df[['Date', 'L', 'currTick', 'currPrice', 'daily_volume', 'TVL']]

In [21]:

# feeTier of pool
feeTier = 3000
# decimals of token0 wbtc
d0 = 8
# decimals of token1 usdc 6 or 18?
d1 = 6
    
# tickSpacing (gaps between two consecutive ticks)
tickSpacing = feeTier / 50

# previous tick of currentTick
df['lowerTick'] = df['currTick'] - ((df['currTick']  % tickSpacing) + tickSpacing) % tickSpacing

# next tick of the currentTick
df['upperTick'] = df['lowerTick'] + tickSpacing

In [22]:
df

,Date,L,currTick,currPrice,daily_volume,TVL,lowerTick,upperTick
0,2021-05-05 08:00:00,1538528889363,63523,57367.030311,2.602096e+05,1.631491e+07,63480.0,63540.0
1,2021-05-06 08:00:00,1540267698477,63368,56484.645158,2.844009e+06,1.590478e+07,63360.0,63420.0
2,2021-05-07 08:00:00,1585095950288,63500,57234.970500,3.494689e+06,1.650661e+07,63480.0,63540.0
3,2021-05-08 08:00:00,1902003547562,63748,58668.300062,4.087189e+06,1.710987e+07,63720.0,63780.0
4,2021-05-09 08:00:00,2139273829361,63669,58209.366317,4.975067e+06,1.767769e+07,63660.0,63720.0
...,...,...,...,...,...,...,...,...
662,2023-02-26 08:00:00,5104602902389,54618,23546.979630,2.150998e+06,5.615007e+07,54600.0,54660.0
663,2023-02-27 08:00:00,5076068119919,54602,23509.807040,3.392830e+06,5.593851e+07,54600.0,54660.0
664,2023-02-28 08:00:00,4248851969393,54435,23120.141637,2.022692e+06,5.559872e+07,54420.0,54480.0
665,2023-03-01 08:00:00,5097678006916,54647,23614.231989,3.548133e+06,5.628960e+07,54600.0,54660.0


In [23]:
# Compute the current price and adjust it to a human-readable format
df['price'] = 1.0001 ** df['currTick']
df['adjusted_price'] = df['price'] / (10 ** (d1 - d0))

# Compute square roots of prices corresponding to the bottom and top ticks
df['sa'] = 1.0001 ** (df['lowerTick']/ 2)
df['sb'] = 1.0001 ** (df['upperTick'] / 2)
df['sp'] = df['price'] ** 0.5

In [24]:


# Compute real amounts of the two assets
df['amount0'] = df['L'] * (df['sb'] - df['sp']) / (df['sp'] * df['sb'])
df['amount1'] = df['L'] * (df['sp'] - df['sa'])



In [25]:
df['adjusted_amount0'] = df['amount0'] / 10 ** d0
df['adjusted_amount1'] = df['amount1'] / 10 ** d1


In [26]:
df['tickTVL'] = df['adjusted_amount0'] * df['currPrice'] + df['adjusted_amount1']

In [27]:
## calculate IV

df['IV'] = 2 * 0.003 * (df['daily_volume'] / df['tickTVL'] * 365) ** 0.5 
df['Implied Volatility'] = df['IV'].apply(lambda x: "{:.0%}".format(x))


In [28]:
## Date as index
df.set_index('Date')

,L,currTick,currPrice,daily_volume,TVL,lowerTick,upperTick,price,adjusted_price,sa,sb,sp,amount0,amount1,adjusted_amount0,adjusted_amount1,tickTVL,IV,Implied Volatility
Date,,,,,,,,,,,,,,,,,,,
2021-05-05 08:00:00,1538528889363,63523,57367.030311,2.602096e+05,1.631491e+07,63480.0,63540.0,573.628748,57362.874777,23.899112,23.970914,23.950548,5.457615e+07,7.913545e+10,0.545761,79135.453609,110444.167584,0.175949,18%
2021-05-06 08:00:00,1540267698477,63368,56484.645158,2.844009e+06,1.590478e+07,63360.0,63420.0,564.806492,56480.649238,23.756154,23.827526,23.765658,1.682804e+08,1.463853e+10,1.682804,14638.530224,109691.124444,0.583683,58%
2021-05-07 08:00:00,1585095950288,63500,57234.970500,3.494689e+06,1.650661e+07,63480.0,63540.0,572.310984,57231.098354,23.899112,23.970914,23.923022,1.323773e+08,3.789944e+10,1.323773,37899.437538,113665.559854,0.635605,64%
2021-05-08 08:00:00,1902003547562,63748,58668.300062,4.087189e+06,1.710987e+07,63720.0,63780.0,586.681030,58668.102955,24.187615,24.260283,24.221499,1.255340e+08,6.444878e+10,1.255340,64448.781642,138097.416529,0.623616,62%
2021-05-09 08:00:00,2139273829361,63669,58209.366317,4.975067e+06,1.767769e+07,63660.0,63720.0,582.064739,58206.473858,24.115164,24.187615,24.126018,2.258113e+08,2.321909e+10,2.258113,23219.085771,154662.396985,0.650137,65%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-26 08:00:00,5104602902389,54618,23546.979630,2.150998e+06,5.615007e+07,54600.0,54660.0,235.456675,23545.667546,15.330794,15.376853,15.344598,6.978276e+08,7.046003e+10,6.978276,70460.034979,234777.354459,0.346968,35%
2023-02-27 08:00:00,5076068119919,54602,23509.807040,3.392830e+06,5.593851e+07,54600.0,54660.0,235.080265,23508.026481,15.330794,15.376853,15.332327,9.586633e+08,7.782016e+09,9.586633,7782.015652,233161.904998,0.437270,44%
2023-02-28 08:00:00,4248851969393,54435,23120.141637,2.022692e+06,5.559872e+07,54420.0,54480.0,231.187216,23118.721646,15.193443,15.239090,15.204842,6.280035e+08,4.843176e+10,6.280035,48431.756034,193627.043796,0.370493,37%


In [29]:
df['minIV'] = df['IV'].rolling(90).min()
df['maxIV'] = df['IV'].rolling(90).max()

In [38]:
df['IV_rank'] = (df['IV'] - df['minIV']) /(df['maxIV'] - df['minIV']) * 10
df = df.dropna()
df['IV_rank'] = df['IV_rank'].apply(lambda x: int(x))

/var/folders/lx/f8gcbnbd18x_k936zw7ddqrh0000gn/T/ipykernel_10239/2121622196.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IV_rank'] = df['IV_rank'].apply(lambda x: int(x))


In [39]:
result = df[['Date','daily_volume', 'TVL', 'tickTVL', 'IV', 'Implied Volatility', 'IV_rank']]
result.to_excel('/Users/wanjinwoo/Desktop/Antalpha/Github/data/IV_wbtcusdc.xlsx')